# FLSim Tutorial: Features to compress client updates for communication efficiency


## Introduction

In this tutorial, we will look at how to use FL Simulator channels to compress client to server messages for communication efficiency. Several approaches, such as quantization, sparsity and sketching have been shown to be useful, and here we guide you through using support in the simulator for three basic techniques, namely half-precision, scalar quantization and Top-K sparsity.

### Prerequisites

Communication efficiency in federated learning is an important problem, as we have to transmit messages between the server (which is usually in a datacenter) and clients, which for example could be mobile devices communicating through a low-bandwidth channel. This causes a bottleneck, specially for the download phase where clients upload model updates to the server. One of the ways this can be addressed is by applying different techniques for compressing the client update. To get started with the motivation for this and get introduced to some of the background techniques, here is a recommended reading list:
- Jakub Konečný, H. Brendan McMahan, Felix X. Yu, Peter Richtarik, Ananda Theertha Suresh, Dave Bacon, "Federated Learning: Strategies for Improving Communication Efficiency", 
NIPS Workshop on Private Multi-Party Machine Learning (2016) https://research.google/pubs/pub45648/
- Sattler, Felix, et al. "Robust and communication-efficient federated learning from non-iid data." IEEE transactions on neural networks and learning systems 31.9 (2019): 3400-3413. https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8889996


Now that you're familiar with the motivation and basic techniques for communication efficiency, lets get started !


### Objectives 

By the end of this tutorial, we will have learnt how to

1. Setup a basic model training pipeline on the Cifar10 task
2. Configure different channels for quantization and sparsity
3. Setup channel measurements for tracking communication
5. Launch FL training flows using FLSim with quantization and sparsity

## Setup a basic training pipeline for the Cifar10 task

### Prerequisite
Let's install flsim via pip with the command below.

In [9]:
!pip install --quiet flsim

Subsequently, let us define the data pipeline and model for training on the Cifar10 task. This is similar to what we have done in the Cifar-10 tutorial at https://github.com/facebookresearch/FLSim/blob/main/tutorials/cifar10_tutorial.ipynb. 

In [10]:
USE_CUDA = True
LOCAL_BATCH_SIZE = 32
EXAMPLES_PER_USER = 500
IMAGE_SIZE = 32

We setup the data pipeline for this task and import the model we will be training on Cifar-10. Similar to the Cifar-10 tutorial, it will be a standard four-layer convolutional neural network, with group norms and a linear layer. We also import the metrics reporter as in the original tutorial.

In [11]:
import torch
from torchvision.datasets.cifar import CIFAR10
from flsim.utils.example_utils import FLModel
from flsim.utils.example_utils import SimpleConvNet

cuda_enabled = torch.cuda.is_available() and USE_CUDA
device = torch.device(f"cuda:{0}" if cuda_enabled else "cpu")

We define a config for training on the Cifar-10 dataset with federated learning. This does not incorporate any settings for client update compression yet, as we will be showing in subsequent sections how to choose a technique and define the configuration for it. We will be modifying this config to incorporate different approaches such as half-precision, scalar quantization and sparsity, and also guide you through their compression parameters.

In [12]:
import flsim.configs
from flsim.utils.config_utils import fl_config_from_json
from omegaconf import OmegaConf


json_config = {
    "trainer": {
        "_base_": "base_sync_trainer",
        # there are different types of aggegator
        # fed avg doesn't require lr, while others such as fed_avg_with_lr or fed_adam do
        "_base_": "base_sync_trainer",
        "server": {
          "_base_": "base_sync_server",
          "server_optimizer": {
            "_base_": "base_fed_avg_with_lr",
            "lr": 2.13,
            "momentum": 0.9
          },
          # type of user selection sampling
          "active_user_selector": {"_base_": "base_uniformly_random_active_user_selector"},
        },
        "client": {
            # number of client's local epoch
            "epochs": 1,
            "optimizer": {
                "_base_": "base_optimizer_sgd",
                # client's local learning rate
                "lr": 0.0138,
                # client's local momentum
                "momentum": 0,
            },
        },
        # number of users per round for aggregation
        "users_per_round": 5,
        # total number of global epochs
        # total #rounds = ceil(total_users / users_per_round) * epochs
        "epochs": 1,
        # frequentcy of reporting train metrics
        "train_metrics_reported_per_epoch": 100,
        # frequency of evaluation per epoch
        "eval_epoch_frequency": 1,
        "do_eval": True,
        # should we report train metrics after global aggregation
        "report_train_metrics_after_aggregation": True,
    }
}

We now define a function *run_cifar10_fl_training()* which runs a round of FL training, and takes as input the FL configuration. We can train under different compression techniques by appropriately changing the config as needed.

In [13]:
from flsim.utils.example_utils import build_data_provider, MetricsReporter

data_provider = build_data_provider(
    local_batch_size=LOCAL_BATCH_SIZE,
    examples_per_user=EXAMPLES_PER_USER,
    image_size = IMAGE_SIZE,
)

from flsim.interfaces.metrics_reporter import Channel

from hydra.utils import instantiate

def run_cifar10_fl_training(json_config):
  model = SimpleConvNet(in_channels=3, num_classes=10)
  global_model = FLModel(model, device)

  if cuda_enabled:
      global_model.fl_cuda()

  cfg = fl_config_from_json(json_config)
  trainer_config = cfg.trainer
  trainer = instantiate(trainer_config, model=global_model, cuda_enabled=cuda_enabled)

  metrics_reporter = MetricsReporter([Channel.TENSORBOARD, Channel.STDOUT])

  final_model, eval_score = trainer.train(
      data_provider=data_provider,
      metric_reporter=metrics_reporter,
      num_total_users=data_provider.num_users(),
      distributed_world_size=1
  )

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


Creating FL User: 100user [00:18,  5.34user/s]
Creating FL User: 20user [00:03,  5.27user/s]
Creating FL User: 20user [00:03,  5.66user/s]

Clients in total: 100


Let us first train a vanilla setting where there is no client update compression, and no metrics reporting. 

In [14]:
run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.34424609541893
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 14.44
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:03<01:07,  3.55s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.319511131942272
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 18.64
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1329794570803644
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 22.08
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:04<00:37,  2.09s/round]

...

Round:  90%|█████████ | 18/20 [00:21<00:02,  1.07s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.2862283729016781
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 55.08
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.240328671783209
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 55.84
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:22<00:01,  1.06s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.329212562739849
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 52.32
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.26285205706954
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 55.12
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.2024490401148795
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 56.56
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, global round = 20), Loss/Eval: 1.263361149840057
(epoch = 1, round = 20, global round = 20), Accuracy/Eval: 55.12


Epoch:   0%|          | 0/1 [00:24<?, ?epoch/s]


## Introduction to communication metrics reporting in FL Simulator

We support tracking of communication metrics through the flag *report_communication_metrics*, and for every round report the mean number of bytes transmitted both for uploads (client to server communication), as well as downloads (server to client communication). So far our focus is mainly on optimizing for upload efficiency as that uploads are generally slower. Let us modify the JSON config above to add a *channel* key to specify channel subconfigs. This controls selection of the compression method, as well as metrics tracking. When *report_communication_metrics* is enabled, the mean upload and download bytes (averaged across participating clients in a round) are reported. Let us consider the simplest compression supported in the simulator, namely the [half-precision channel](https://github.com/facebookresearch/FLSim/blob/main/channels/half_precision_channel.py). Here, client updates (which by default are 32-bit floating points) are compressed to 16-bit. We can specify the compression method with the *_base_* key here in the sub-config.

In [15]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_half_precision_channel"
}
cfg = fl_config_from_json(json_config)


/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."


After defining the JSON config and converting to an FL config, we instantiate the trainer and start training with half-precision compression. While the server sends full-precision models to clients, the client uploads consume half the number of bytes as expected.

In [16]:
run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.3217339515686035
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 13.08
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:20,  1.09s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.254027526080608
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 21.64
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1212334811687468
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 25.16
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:19,  1.10s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.08s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.339336696267128
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 51.72
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.2747029773890972
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 54.92
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:20<00:01,  1.08s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.3354713223874568
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 51.88
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.3052193865180015
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 53.16
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.2794485621154308
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 54.64
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 63188.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, 

Epoch:   0%|          | 0/1 [00:22<?, ?epoch/s]


We now look at the scalar quantization channel, which quantizes tensors in ecah client update to between 1 and 8 bits. The channel supports both *per-tensor quantization* and *per-channel quantization*. Let us consider the former which is the simplest case, where each tensor shares the same zero-point and scale parameters. The channel configuration can be now modified to select this channel, with its associated per-tensor quantization flag and number of bits (*n_bits*) parameter.

In [17]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_scalar_quantization_channel",
  "n_bits": 3,
  "quantize_per_tensor": True
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.3254137188196182
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 13.6
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:21,  1.13s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.3311036705970762
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 18.32
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.134718155860901
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 22.72
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:19,  1.11s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.10s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.248136217147112
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 55.64
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.28172812089324
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 54.0
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.10s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.3301015309989452
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 52.96
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.2673112235963344
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 54.24
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.2815353155136109
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 53.48
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 12640.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, 

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


Note how the number of bytes during upload is not exactly 11847 (=126376 * 3/32). The additional overhead of 793 bytes is due to the upload messages also including scale and zero-point information, which is necessary server-side to dequantize the client update tensors. We now wrap up this tutorial by looking at another channel, [sparse mask channel](https://github.com/facebookresearch/FLSim/blob/main/channels/sparse_mask_channel.py) which implements sparsity on the client updates. The sparsity mask is client-specific, and the simulator supports two approaches to determine this mask : *random* and *Top-K*. Random mask is self-explanatory, with the per-tensor sparsity mask being set randomly based on the sparsity level, and *Top-K* sparsity masking out the lowest magnitude elements within each model update tensor. Let us now define a config using the Top-K mask with a sparsity level of 95%, and run a round of training using this compression. 

In [18]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_sparse_mask_channel",
  "proportion_of_zero_weights":0.95,
  "sparsity_method": "topk"
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.355492594838142
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 12.48
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:21,  1.12s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.245325019955635
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 16.68
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 10285.25
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.1523214742541312
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 20.68
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:20,  1.12s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.10s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.3205247521400452
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 51.84
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 10285.25
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.2996930867433547
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 53.04
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.11s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.2829436622560024
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 54.6
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 10285.25
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.2664362393319606
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 54.76
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.2829885132610799
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 52.8
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 10285.25
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, 

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


While the non-sparse parameters consume only 37912 bytes, we also need to pack in additional information about the sparsity pattern per tensor. The simulator supports two ways to do this: *bitmask* and *coo*. In the bitmask setting, the sparsity pattern is transported as a concatenated bitstream. We also support another mode *coo*, which follows the popular COO format for encoding sparse tensors. Here, the non-sparse tensor weights are transmitted along with their 64-bit integer coordinates. Generally *bitmask* has a lower overhead unless at very high sparsity values, where *coo* is more efficient. Let us conclude our tutorial by running a round of Cifar-10 training with *coo* bytes measurement.

In [19]:
json_config["trainer"]["channel"] = {
  "report_communication_metrics": True,
  "_base_": "base_sparse_mask_channel",
  "proportion_of_zero_weights": 0.95,
  "sparsity_method": "topk",
  "compressed_size_measurement": "coo"
}

run_cifar10_fl_training(json_config)

/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
Round:   0%|          | 0/20 [00:00<?, ?round/s]

Train finished Global Round: 1
(epoch = 1, round = 1, global round = 1), Loss/Training: 2.2945614755153656
(epoch = 1, round = 1, global round = 1), Accuracy/Training: 15.6
reporting (epoch = 1, round = 1, global round = 1) for aggregation


Round:   5%|▌         | 1/20 [00:01<00:20,  1.10s/round]

(epoch = 1, round = 1, global round = 1), Loss/Aggregation: 2.2425229862332343
(epoch = 1, round = 1, global round = 1), Accuracy/Aggregation: 13.64
(epoch = 1, round = 1, global round = 1), Client to Server Bytes Sent/Training: 54408.0
(epoch = 1, round = 1, global round = 1), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 2
(epoch = 1, round = 2, global round = 2), Loss/Training: 2.0917357608675955
(epoch = 1, round = 2, global round = 2), Accuracy/Training: 23.44
reporting (epoch = 1, round = 2, global round = 2) for aggregation


Round:  10%|█         | 2/20 [00:02<00:19,  1.10s/round]

...

Round:  90%|█████████ | 18/20 [00:19<00:02,  1.11s/round]

(epoch = 1, round = 18, global round = 18), Loss/Aggregation: 1.2449420608580113
(epoch = 1, round = 18, global round = 18), Accuracy/Aggregation: 54.88
(epoch = 1, round = 18, global round = 18), Client to Server Bytes Sent/Training: 54408.0
(epoch = 1, round = 18, global round = 18), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 19
(epoch = 1, round = 19, global round = 19), Loss/Training: 1.2409622445702553
(epoch = 1, round = 19, global round = 19), Accuracy/Training: 54.8
reporting (epoch = 1, round = 19, global round = 19) for aggregation


Round:  95%|█████████▌| 19/20 [00:21<00:01,  1.11s/round]

(epoch = 1, round = 19, global round = 19), Loss/Aggregation: 1.264750774204731
(epoch = 1, round = 19, global round = 19), Accuracy/Aggregation: 54.24
(epoch = 1, round = 19, global round = 19), Client to Server Bytes Sent/Training: 54408.0
(epoch = 1, round = 19, global round = 19), Server to Client Bytes Sent/Training: 126376.0
Train finished Global Round: 20
(epoch = 1, round = 20, global round = 20), Loss/Training: 1.1887283816933631
(epoch = 1, round = 20, global round = 20), Accuracy/Training: 56.52
reporting (epoch = 1, round = 20, global round = 20) for aggregation
(epoch = 1, round = 20, global round = 20), Loss/Aggregation: 1.1686582043766975
(epoch = 1, round = 20, global round = 20), Accuracy/Aggregation: 59.04
(epoch = 1, round = 20, global round = 20), Client to Server Bytes Sent/Training: 54408.0
(epoch = 1, round = 20, global round = 20), Server to Client Bytes Sent/Training: 126376.0
Running (epoch = 1, round = 20, global round = 20) for Eval
(epoch = 1, round = 20, g

Epoch:   0%|          | 0/1 [00:23<?, ?epoch/s]


This tutorial has provided a concise overview of how to use the existing channel features in FLSim for communication efficiency. This should be useful for any development work on experimenting with different compression approaches to enable scalable federated learning. Please refer to https://github.com/facebookresearch/FLSim/tree/main/channels for the code pointers to implementation of different channels as well as the communication metrics reporting.